In [2]:
import torch
from torch import nn
torch.manual_seed(0)
import random
random.seed(0)
from transformers import BertTokenizer, BertModel
from util import *
from modules import *
from loss_functions import *
from models import *
from loss_functions import *
import numpy as np
import sys
import time

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import umap
%matplotlib inline
import umap.plot
import json

In [58]:
import pandas as pd

In [7]:
config = json.load(open('configs/model1_config.json','r'))
modal_idx = {modal: config['modals'].index(modal) for modal in config['modals']}
device = "cuda:2" if torch.cuda.is_available() else "cpu"
idx_modal = {key:value for value, key in modal_idx.items()}

In [8]:
generator = joint_align(config['mode'], 
                        modals=config['modals'],
                        modal_idx=modal_idx, 
                        token_dim=config['generator']['token_dim'], 
                        sense_dim=config['generator']['sense_dim'], 
                        hid_dim=config['generator']['hid_dim'],
                        k_assignment=config['k_assignment']).to(device)

In [9]:
model_path = "model_checkpoints/sense_level_1/joint_align_1.pth"
generator.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [10]:
modal_sents = {modal: get_sents_from_file("modal_sentences/test/{}.txt".format(modal)) for modal in generator.modals}

# Evaluation

to examine the model's performance, we need to get the projected sense embeddings and the sense centriods for each modal word and visualize them all together.

In [11]:
'''
batch_embeddings = {modal: get_bert_embeddings(tokenizer,bert,sents,modal,device)[0] for modal, sents in batch_sentences.items()}
'''
bert = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
embedding_sentences = {modal: get_bert_embeddings(tokenizer,bert,sents,modal,device) for modal, sents in modal_sents.items()}

100%|██████████| 512/512 [00:45<00:00, 11.18it/s]


In [13]:
embedding_tensors = {modal: value[0] for modal, value in embedding_sentences.items()}
sentences = {modal: value[1] for modal, value in embedding_sentences.items()}

In [16]:
gen_out = generator(embedding_tensors, device)

In [17]:
gen_out.keys()

dict_keys(['sense_embeddings', 'labels', 'segments', 'embedding_dict', 'centroid_dict', 'distribution_dict'])

In [14]:
gen_out['centroid_dict']

{'can': tensor([[ 3.2748,  1.2635, -5.1753,  ..., -2.6800,  6.5355,  9.8724],
         [-0.6297, -0.7398,  1.1867,  ...,  0.4459, -1.3283, -2.5102],
         [-1.0115, -0.9268,  1.7846,  ...,  0.8028, -2.1374, -3.7086]],
        device='cuda:2', grad_fn=<DivBackward0>),
 'might': tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [ -1.7168, -15.4667,   4.4352,  ...,   0.7305,  -5.8824, -12.5265],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]],
        device='cuda:2', grad_fn=<DivBackward0>),
 'should': tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [  9.0960,  -2.4426,  12.9123,  ...,  13.6404, -14.8347, -18.8891]],
        device='cuda:2', grad_fn=<DivBackward0>),
 'must': tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
         [ 10.8758,  11.6725, -12.3269,  ...,   3.8545,   6.3

In [22]:
gen_out['distribution_dict']

{'can': tensor([[1.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000],
         ...,
         [1.0000, 0.0000, 0.0000],
         [0.0013, 0.0023, 0.9964],
         [1.0000, 0.0000, 0.0000]], device='cuda:2', grad_fn=<SoftmaxBackward>),
 'might': tensor([[0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         ...,
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.]], device='cuda:2', grad_fn=<SoftmaxBackward>),
 'should': tensor([[0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         ...,
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.]], device='cuda:2', grad_fn=<SoftmaxBackward>),
 'must': tensor([[0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.],
         ...,
         [0., 1., 0.],
         [0., 1., 0.],
         [0., 1., 0.]], device='cuda:2', grad_fn=<SoftmaxBackward>)}

In [59]:
sense_embeddings = gen_out['embedding_dict']
centroid_dict = gen_out['centroid_dict']
distribution_dict = gen_out['distribution_dict']
sentences = {modal: value[1] for modal, value in embedding_sentences.items()}

_labels = list()
_embeddings = list()
_sentences = list()

for modal in sense_embeddings.keys():
    embs = sense_embeddings[modal].cpu().detach().numpy()
    dist = distribution_dict[modal].cpu().detach().numpy()
    labels = np.argmax(dist, axis = 1)
    str_labels = ["{}_{}".format(modal,label) for label in labels]
    _labels += str_labels
    if len(_embeddings) == 0:
        _embeddings = embs
    else:
        _embeddings = np.append(_embeddings, embs,axis = 0)
    
    _sentences += sentences[modal]
    
hover_data = pd.DataFrame({'index':np.arange(len(_labels)),
                           'label':_labels,
                           'sentences':_sentences})
    

In [60]:
umap.plot.output_notebook()

Loading BokehJS ...

In [64]:
mapper = umap.UMAP(n_neighbors=15).fit(_embeddings)
p = umap.plot.interactive(mapper, labels=_labels, hover_data=hover_data, point_size=5)
umap.plot.show(p)